In [17]:
import pygame
import torch
import import_ipynb
from player import *

In [18]:
# Constants
WIDTH, HEIGHT = 1200,700
BOARD_SIDE = 600
EXTRA = (1200 - BOARD_SIDE) // 2
GRID_SIZE = 8
CELL_SIZE = BOARD_SIDE // GRID_SIZE
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 128, 0)


# Initialize the game board
board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
board[3][4] = board[4][3] = 2  # White
board[3][3] = board[4][4] = 1  # Black

# Current player (1 for White, 2 for Black)


# Initialize Pygame
# # Create the game window

pygame.init()
pygame.font.init() 
my_font = pygame.font.SysFont('Comic Sans MS', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Othello")

def draw_board(prev_move):
    screen.fill(WHITE)
    pygame.draw.rect(screen, GREEN, (EXTRA ,0, BOARD_SIDE, BOARD_SIDE),0)
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if (x,y) == prev_move: pygame.draw.rect(screen, (240,100,100), (EXTRA + x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 0)
            pygame.draw.rect(screen, BLACK, (EXTRA + x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE), 1)
            if board[y][x] == 1:
                pygame.draw.circle(screen, WHITE, (EXTRA + x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)
                pygame.draw.circle(screen, BLACK, (EXTRA + x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5,1)
            elif board[y][x] == 2:
                pygame.draw.circle(screen, BLACK, (EXTRA + x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 5)


In [19]:
current_player = 2

P1 = 5
P2 = 5

def inRect(X,Y,WIDTH,HEIGHT,x,y):
    if X <= x <= X + WIDTH and Y <= y <= Y + HEIGHT: return True
    return False

In [20]:
players = [
    RLMCTSPlayer(timeLimit = 2.0, timed = True),
    MCTSPlayer(timeLimit = 2.0,timed=True),
    MCTSParallelPlayer(timeLimit = 2.0,timed = True),
    AlphaBetaPlayer(depth=3),
    RandomPlayer()
]

players[0].policy_value_network.load_state_dict(torch.load('RLMCTS_5SEC_1.4/checkpoint19.pth', map_location=torch.device("cpu"),weights_only=True))

R_WIDTH,R_HEIGHT = 200,80

def draw_selector(screen,X,Y,current_select):

    player_names = [
        "RLMCTS",
        "MCTS",
        "MCTSParallel",
        "AlphaBeta",
        "Random",
        "Human"
    ]

    for i in range(len(player_names)):
        if i == current_select: pygame.draw.rect(screen,(200,200,200),(X,Y + i*R_HEIGHT,R_WIDTH,R_HEIGHT),0)
        else: pygame.draw.rect(screen,(230,230,230),(X,Y + i*R_HEIGHT,R_WIDTH,R_HEIGHT),0)

        text_surface = my_font.render(player_names[i], True, BLACK)
        text_rect = text_surface.get_rect(topleft= (X,Y + i*R_HEIGHT))
        screen.blit(text_surface, text_rect)

def select_selector(X,Y,x,y):
    for i in range(6):
        if inRect(X,Y + R_HEIGHT*i,R_WIDTH,R_HEIGHT,x,y) : return i
    return None
        


In [21]:
prev_move = None

def draw_valid(screen,board,current_player):
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player):
                pygame.draw.circle(screen, (200,200,0), (EXTRA + x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 30)
                pygame.draw.circle(screen, BLACK, (EXTRA + x*CELL_SIZE + CELL_SIZE//2, y*CELL_SIZE + CELL_SIZE//2), CELL_SIZE//2 - 30,1)

def print_stats(screen,board,current_player):
    WHITE = 0
    BLACK = 0
    for x in board:
        for y in x:
            WHITE += y == 1
            BLACK += y == 2
    text_surface = my_font.render(f"White : {WHITE}p", True, BLACK)
    text_rect = text_surface.get_rect(topleft= (80,530))
    screen.blit(text_surface, text_rect)
    text_surface = my_font.render(f"Black : {BLACK}p", True, BLACK)
    text_rect = text_surface.get_rect(topleft= (980,530))
    screen.blit(text_surface, text_rect)

    text_surface = my_font.render(f"{'Black' if current_player == 2 else 'White'}'s Turn", True, BLACK)
    text_rect = text_surface.get_rect(topleft= (510,620))
    screen.blit(text_surface, text_rect)
                

In [22]:
while True:
    Quit = False
    for event in pygame.event.get():
        
        if P1 != 5 and current_player == 1:
            x,y = (players[P1]).get_move(board,current_player)    
            if is_valid_move(x, y,board,current_player):
                make_move(x,y,board,current_player)
                prev_move = (x,y)
            current_player = 2
        elif P2 != 5 and current_player == 2:
            x,y = (players[P2]).get_move(board,current_player)    
            if is_valid_move(x, y,board,current_player):
                make_move(x,y,board,current_player)
                prev_move = (x,y)
            current_player = 1
        
        if event.type == pygame.QUIT:
            Quit = True
        if event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            if select_selector(50,30,x,y) != None: P1 = select_selector(50,30,x,y)
            if select_selector(950,30,x,y) != None: P2 = select_selector(950,30,x,y)
            if inRect(EXTRA,0,BOARD_SIDE,BOARD_SIDE,x,y):
                if event.type == pygame.MOUSEBUTTONDOWN and current_player == 1 and P1 == 5:
                    x, y = event.pos
                    x, y = (x - EXTRA) // CELL_SIZE, y // CELL_SIZE
                    if is_valid_move(x, y,board,current_player):
                        make_move(x,y,board,current_player)
                        prev_move = (x,y)
                        current_player = 2
                elif event.type == pygame.MOUSEBUTTONDOWN and current_player == 2 and P2 == 5:
                    x, y = event.pos
                    x, y = (x - EXTRA) // CELL_SIZE, y // CELL_SIZE
                    if is_valid_move(x, y,board,current_player):
                        make_move(x,y,board,current_player)
                        prev_move = (x,y)
                        current_player = 1

        if is_game_over(board,current_player):
            board = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
            board[3][4] = board[4][3] = 2  # White
            board[3][3] = board[4][4] = 1  # Black
            prev_move = None
            current_player = 2
            
    if(Quit): break

    draw_board(prev_move)
    if P1 == 5 and current_player == 1: draw_valid(screen,board,1)
    if P2 == 5 and current_player == 2: draw_valid(screen,board,2)
    draw_selector(screen,50,30,P1)
    draw_selector(screen,950,30,P2)
    print_stats(screen,board,current_player)
    pygame.display.flip()
    
pygame.quit()